# Setup

In [1]:
import os

from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [2]:
import phoenix as px

session = px.launch_app()

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [3]:
from phoenix.trace.langchain import LangChainInstrumentor

LangChainInstrumentor().instrument()

# Ingest Data

In [4]:
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter

In [5]:
files = os.listdir("../city_data")
file_texts = []
for file in files:
    with open(f"../city_data/{file}") as f:
        file_text = f.read()
    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=1234,
        chunk_overlap=123,
    )
    texts = text_splitter.split_text(file_text)
    for i, chunked_text in enumerate(texts):
        file_texts.append(
            Document(
                page_content=chunked_text,
                metadata={"doc_title": file.split(".")[0], "chunk_num": i},
            )
        )

WARNI [langchain_text_splitters.base] Created a chunk of size 1311, which is longer than the specified 1234
WARNI [langchain_text_splitters.base] Created a chunk of size 1280, which is longer than the specified 1234
WARNI [langchain_text_splitters.base] Created a chunk of size 2076, which is longer than the specified 1234
WARNI [langchain_text_splitters.base] Created a chunk of size 1344, which is longer than the specified 1234


In [6]:
from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain.vectorstores import FAISS

In [7]:
embeddings = HuggingFaceEmbeddings()

/Users/nicktroast/Workspace/rag_cookbooks/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/Users/nicktroast/Workspace/rag_cookbooks/.venv/lib/python3.12/site-packages/sentence_transformers/evaluation/SentenceEvaluator.py:81: SyntaxWarning: invalid escape sequence '\g'
  return re.sub(r"([a-z])([A-Z])", "\g<1> \g<2>", class_name)
/Users/nicktroast/Workspace/rag_cookbooks/.venv/lib/python3.12/site-packages/sentence_transformers/model_card.py:524: SyntaxWarning: invalid escape sequence '\d'
  if dataset_name and re.match("_dataset_\d+", dataset_name):
/Users/nicktroast/Workspace/rag_cookbooks/

In [8]:
vector_store = FAISS.from_documents(file_texts, embedding=embeddings)

# Search the Data

In [10]:
from langchain_community.llms import Ollama

llm = Ollama(
    model="llama3",
    temperature=0.1,
    top_p=0.9,
)

In [11]:
retriever = vector_store.as_retriever()

In [12]:
from langchain.prompts import ChatPromptTemplate

template = """You are a tour guide. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [16]:
chain.invoke("Does Seattle have a basketball team?")

"What a great question! As a tour guide, I'm excited to share with you the significant musical history of Seattle.\n\nSeattle has a rich jazz heritage, and between 1918 and 1951, nearly 24 jazz nightclubs thrived along Jackson Street. This vibrant scene nurtured the early careers of legendary musicians like Ernestine Anderson, Ray Charles, Quincy Jones, and others. The city's jazz legacy continues to influence music today.\n\nBut that's not all! Seattle is also the birthplace of several iconic rock bands, including Foo Fighters, Heart, and Jimi Hendrix. And let's not forget the grunge movement, which originated in the late 20th century with pioneering bands like Alice in Chains, Nirvana, Pearl Jam, Soundgarden, and more.\n\nSo, if you're a music lover, Seattle is definitely a city worth exploring!"

# Run Evals

In [17]:
from phoenix.evals import (
    HallucinationEvaluator,
    OpenAIModel,
    QAEvaluator,
    RelevanceEvaluator,
    run_evals,
)
from phoenix.session.evaluation import get_qa_with_reference, get_retrieved_documents
from phoenix.trace import DocumentEvaluations, SpanEvaluations

In [18]:
queries_df = get_qa_with_reference(px.Client())
retrieved_documents_df = get_retrieved_documents(px.Client())

In [19]:
eval_model = OpenAIModel(
    model="llama3",
    base_url="http://localhost:11434/v1",
)
hallucination_evaluator = HallucinationEvaluator(eval_model)
qa_correctness_evaluator = QAEvaluator(eval_model)
relevance_evaluator = RelevanceEvaluator(eval_model)

hallucination_eval_df, qa_correctness_eval_df = run_evals(
    dataframe=queries_df,
    evaluators=[hallucination_evaluator, qa_correctness_evaluator],
    provide_explanation=True,
)
relevance_eval_df = run_evals(
    dataframe=retrieved_documents_df,
    evaluators=[relevance_evaluator],
    provide_explanation=True,
)[0]

px.Client().log_evaluations(
    SpanEvaluations(eval_name="Hallucination", dataframe=hallucination_eval_df),
    SpanEvaluations(eval_name="QA Correctness", dataframe=qa_correctness_eval_df),
    DocumentEvaluations(eval_name="Relevance", dataframe=relevance_eval_df),
)

WARNI [phoenix.evals.executors] 🐌!! If running llm_classify inside a notebook, patching the event loop with nest_asyncio will allow asynchronous eval submission, and is significantly faster. To patch the event loop, run `nest_asyncio.apply()`.


run_evals |          | 0/6 (0.0%) | ⏳ 00:00<? | ?it/s

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
WARNI [phoenix.evals.executors] 🐌!! If running llm_classify inside a notebook, patching the event loop with nest_asyncio will allow asynchronous eval submission, and is significantly faster. To patch the event loop, run `nest_asyncio.apply()`.


run_evals |          | 0/12 (0.0%) | ⏳ 00:00<? | ?it/s